# Setup

# Setup

## Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


## Installs

In [ ]:
!pip install pyserini -q
!pip install faiss-cpu -q
!pip install jsonlines -q
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 35.6 MB/s eta 0:00:00


## Paths

In [ ]:
DATASET_FILE = './expanded_docs.jsonl'

In [ ]:
!cp './gdrive/Shareddrives/UNICAMP/DOUTORADO/IA368/[5] Doc2Query/expanded_docs.jsonl' . 

# Helper Function

In [ ]:
from tqdm.auto import tqdm
import json

def get_documents(expanded=False):
  with open(DATASET_FILE, 'r') as fp:
    for line in fp:
      line_dict = json.loads(line)

      del line_dict['queries']
      line_dict['id'] = line_dict["_id"]
      del line_dict["_id"]

      if expanded:
        del line_dict['text']
        line_dict['contents'] = line_dict["expanded_text"]
        del line_dict["expanded_text"]
        del line_dict["title"]

        yield line_dict
      else:
        line_dict['contents'] = line_dict['title'] + ' ' + line_dict['text']
        del line_dict['text']
        del line_dict["expanded_text"]
        del line_dict["title"]
        yield line_dict

# Qrels

In [ ]:
from datasets import load_dataset

qrels_dataset = load_dataset("BeIR/trec-covid-qrels")
qrels_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/BeIR___csv/BeIR--trec-covid-qrels-1766e3af5b0b856a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

# Queries

In [ ]:
queries_dataset = load_dataset("BeIR/trec-covid", "queries")
queries_dataset

Generating queries split: 0 examples [00:00, ? examples/s]

Dataset trec-covid downloaded and prepared to /root/.cache/huggingface/datasets/BeIR___trec-covid/queries/0.0.0/093f1fe2ffa7a9c72fa48239c8f279b51d6b171abd77737c7fd1406125307599. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    queries: Dataset({
        features: ['_id', 'title', 'text'],
        num_rows: 50
    })
})

# BM25 - Plain Doc

In [ ]:
!mkdir -p collections/non_expanded

In [ ]:
import jsonlines

for non_expanded_doc in get_documents(expanded=False):
  with jsonlines.open(f"collections/non_expanded/non_expanded_docs.jsonl", 'a') as writer:
      writer.write(non_expanded_doc)  

## Build Index

In [ ]:
%%time
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input collections/non_expanded/ \
  --index indexes/lucene-index-trec-covid-non-expanded \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

2023-04-10 18:52:02,931 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-04-10 18:52:02,934 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-04-10 18:52:02,935 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-04-10 18:52:02,935 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: collections/non_expanded/
2023-04-10 18:52:02,936 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-04-10 18:52:02,936 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-04-10 18:52:02,937 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 9
2023-04-10 18:52:02,937 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: en
2023-04-10 18:52:02,938 INFO  [main] index.IndexCollecti

In [ ]:
from pyserini.index import IndexReader

reader = IndexReader('indexes/lucene-index-trec-covid-non-expanded')
total_terms = reader.stats()["total_terms"]
print(f"Total number of entries in the index: {total_terms:,}")

Total number of entries in the index: 20,822,821


## Rank

In [ ]:
from pyserini.search import get_topics, get_qrels, SimpleSearcher
from tqdm.auto import tqdm
from pyserini.search.lucene import LuceneSearcher
from collections import defaultdict

searcher = LuceneSearcher('indexes/lucene-index-trec-covid-non-expanded')
searcher.set_bm25(k1=0.9, b=0.4)

scored_output = defaultdict(list)

for query in tqdm(queries_dataset['queries'], desc='Running Queries'):
  hits = searcher.search(query['text'], 1000)

  for i, hit in enumerate(hits):
    scored_output[query["_id"]].append({'doc_id': hit.docid,
                                        'bm25_score': hit.score,
                                        'bm25_position': i
                                        })

Running Queries:   0%|          | 0/50 [00:00<?, ?it/s]

## Eval

In [ ]:
!pip install trectools -q
!pip install evaluate -q

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.3 MB/s eta 0:00:00


In [ ]:
qrels_format = defaultdict(list)

for query in qrels_dataset['test']:
  qrels_format['query'].append(query["query-id"])
  qrels_format['q0'].append("q0")
  qrels_format['docid'].append(str(query["corpus-id"]))
  qrels_format['rel'].append(query["score"])

In [ ]:
run_format = defaultdict(list)

for query_id in scored_output:
  results = scored_output[query_id]
  rank = 1
  for result in results:
    run_format['query'].append(query_id)
    run_format['q0'].append("q0")
    run_format['docid'].append(str(result['doc_id']))
    run_format['rank'].append(rank)
    run_format['score'].append(result['bm25_score'])
    run_format['system'].append("Reranker")    

    rank += 1

In [ ]:
from evaluate import load
trec_eval = load("trec_eval")

In [ ]:
results = trec_eval.compute(predictions=[run_format], references=[qrels_format])

In [ ]:
results['NDCG@10']

0.5946917010118077

# BM25 - Expanded Docs

In [ ]:
!mkdir -p collections/expanded

In [ ]:
import jsonlines

for non_expanded_doc in get_documents(expanded=True):
  with jsonlines.open(f"collections/expanded/expanded_docs.jsonl", 'a') as writer:
      writer.write(non_expanded_doc)  

## Build Index

In [ ]:
%%time
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input collections/expanded/ \
  --index indexes/lucene-index-trec-covid-expanded \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

2023-04-10 19:14:33,212 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-04-10 19:14:33,221 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-04-10 19:14:33,221 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-04-10 19:14:33,222 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: collections/expanded/
2023-04-10 19:14:33,222 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-04-10 19:14:33,223 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-04-10 19:14:33,223 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 9
2023-04-10 19:14:33,224 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: en
2023-04-10 19:14:33,224 INFO  [main] index.IndexCollection (

## Rank

In [ ]:
from pyserini.search import get_topics, get_qrels, SimpleSearcher
from tqdm.auto import tqdm
from pyserini.search.lucene import LuceneSearcher
from collections import defaultdict

searcher = LuceneSearcher('indexes/lucene-index-trec-covid-expanded')
searcher.set_bm25(k1=0.9, b=0.4)

scored_output = defaultdict(list)

for query in tqdm(queries_dataset['queries'], desc='Running Queries'):
  hits = searcher.search(query['text'], 1000)

  for i, hit in enumerate(hits):
    scored_output[query["_id"]].append({'doc_id': hit.docid,
                                        'bm25_score': hit.score,
                                        'bm25_position': i
                                        })

Running Queries:   0%|          | 0/50 [00:00<?, ?it/s]

## Eval

In [ ]:
run_format = defaultdict(list)

for query_id in scored_output:
  results = scored_output[query_id]
  rank = 1
  for result in results:
    run_format['query'].append(query_id)
    run_format['q0'].append("q0")
    run_format['docid'].append(str(result['doc_id']))
    run_format['rank'].append(rank)
    run_format['score'].append(result['bm25_score'])
    run_format['system'].append("Reranker")    

    rank += 1

In [ ]:
from evaluate import load
trec_eval = load("trec_eval")

In [ ]:
results = trec_eval.compute(predictions=[run_format], references=[qrels_format])

In [ ]:
results['NDCG@10']

0.6556285378554335